In [ ]:
! pip install transformers

     |████████████████████████████████| 1.8MB 18.8MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
     |████████████████████████████████| 2.9MB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9ac942a0f6c33a2f09541477a92bf514a247136200f4cb6b83e770dc36adfa07
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
api_token = {"username":"kangchunglin","key":"0c9c8e18dffb7a6b29f7f6f4d12fe075"}
import json
import zipfile
import os
 
if not os.path.exists("/root/.kaggle"):
    os.makedirs("/root/.kaggle")
 
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json
 
if not os.path.exists("/kaggle"):
    os.makedirs("/kaggle")
os.chdir('/kaggle')
!kaggle competitions download -c nlp-getting-started
 
!ls /kaggle

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 65.9MB/s]
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 7.36MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 127MB/s]
sample_submission.csv  test.csv  train.csv


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split
from transformers import *

In [ ]:
"""
載入預訓練模型
"""
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
"""
載入模型斷詞工具
"""
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_data=pd.read_csv('../kaggle/train.csv')
test_data=pd.read_csv('../kaggle/test.csv')

In [ ]:
train_data.drop(['keyword','location'], inplace=True, axis=1)
train_data.columns=['idx','sentence','label']

In [ ]:
train_data.head()

,idx,sentence,label
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
"""
訓練集80%，測試集20%
"""
train_size = 0.8

mask = np.random.rand(len(train_data)) < train_size
train_dataset = train_data[mask]
valid_dataset = train_data[~mask]

In [ ]:
train_size = len(train_dataset)
valid_size = len(valid_dataset)
print('Train size: ', train_size)
print('Valid size: ', valid_size)

Train size:  6107
Valid size:  1506


In [ ]:
train_dataset1 = tf.data.Dataset.from_tensor_slices(dict(train_dataset))
valid_dataset1 = tf.data.Dataset.from_tensor_slices(dict(valid_dataset))

In [ ]:
max_length = 64
task = 'cola' #二元分類

train_dataset = glue_convert_examples_to_features(train_dataset1,
                          tokenizer,
                          max_length,
                          task,label_list =['0','1'])
valid_dataset = glue_convert_examples_to_features(valid_dataset1,
                          tokenizer,
                          max_length,
                          task,label_list =['0','1'])

/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:67: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)
/usr/local/lib/python3.6/dist-packages/transformers/data/processors/glue.py:284: FutureWarning: This processor will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("processor"), FutureWarning)


In [ ]:
train_temp = next(iter(train_dataset))

In [ ]:
train_temp

({'attention_mask': <tf.Tensor: shape=(64,), dtype=int32, numpy=
  array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        dtype=int32)>,
  'input_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
  array([  101,  2256, 15616,  2024,  1996,  3114,  1997,  2023,  1001,
          8372,  2089, 16455,  9641,  2149,  2035,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0], dtype=int32)>,
  'token_type_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
  array([0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
buffer_size = 100
train_bz = 6
epochs = 3
valid_bz = 6

train_gen = train_dataset.shuffle(buffer_size).batch(train_bz).repeat(epochs)
valid_gen = valid_dataset.batch(valid_bz)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5,
                    epsilon=1e-8,
                    clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                            reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
history = model.fit(train_gen,
           epochs=epochs,
           steps_per_epoch=train_size//train_bz, 
           validation_data=valid_gen,
           validation_steps=valid_size//valid_bz)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f0944f9fd90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f0944f9fd90> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1017/1017 [==============================] - ETA: 0s - loss: 0.4925 - accuracy: 0.7707

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1017/1017 [==============================] - 157s 117ms/step - loss: 0.4924 - accuracy: 0.7707 - val_loss: 0.3872 - val_accuracy: 0.8446
Epoch 2/3
1017/1017 [==============================] - 122s 120ms/step - loss: 0.3468 - accuracy: 0.8652 - val_loss: 0.4336 - val_accuracy: 0.8466
Epoch 3/3
1017/1017 [==============================] - 123s 121ms/step - loss: 0.2536 - accuracy: 0.9087 - val_loss: 0.5202 - val_accuracy: 0.8327


In [ ]:
# save_path = 'save'
# if not os.path.exists(save_path):
#     os.mkdir(save_path)
# model.save_pretrained('./save/')

In [ ]:
valid_pred = model.predict(valid_gen)
valid_pred_ids = np.argmax(valid_pred[0], axis=-1)

In [ ]:
valid_label = []
for x in valid_dataset:
    valid_label += [x[1].numpy()]

In [ ]:
print(classification_report(y_pred=valid_pred_ids, y_true=valid_label))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       851
           1       0.81      0.80      0.81       655

    accuracy                           0.83      1506
   macro avg       0.83      0.83      0.83      1506
weighted avg       0.83      0.83      0.83      1506



In [ ]:
confm = confusion_matrix(y_pred=valid_pred_ids, y_true=valid_label)

index = ['Actual_0', 'Actual_1']
columns = ['Pred_0', 'Pred_1']
pd.DataFrame(confm, index=index, columns=columns)

,Pred_0,Pred_1
Actual_0,732,119
Actual_1,133,522


### **情感分析**

In [ ]:
! pip install transformers

In [25]:
import numpy as np
import pandas as pd

import torch

# 將字詞轉換成token id
from transformers import BertTokenizer
# 引入pretrain model
from transformers import BertForSequenceClassification
# 建立torch的dataset
from torch.utils.data import TensorDataset
# pytorch須將資料放在DataLoader
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# 與optimizer有關
from transformers import AdamW, get_linear_schedule_with_warmup
# 評估模型
from sklearn.metrics import f1_score
# 輸出畫面優化
from tqdm.notebook import tqdm

In [14]:
# read files
df_train = pd.read_csv('../content/train.txt', 
             header =None, sep =';', names = ['sentence','label'], encoding='utf-8')
df_val = pd.read_csv('../content/val.txt', 
            header = None, sep =';', names = ['sentence','label'],encoding='utf-8')

In [15]:
df_train.head()

,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [16]:
max([len(s) for s in df_train.sentence])

300

In [17]:
# 將類別給予編號
encoding = { 'anger': 0,
    'fear': 1,
    'joy': 2,
    'love': 3,
    'sadness': 4,
    'surprise': 5
}

In [18]:
# 將類別名稱轉換成編號
df_train['label'] = [encoding[key] for key in df_train.label]
df_val['label'] = [encoding[key] for key in df_val.label]

In [19]:
# download tokenizer的模型 bert-base(large)-uncased(chinese)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# 將text轉換成 token_id, attention_mask                                           
encoded_data_train = tokenizer.batch_encode_plus(
    df_train.sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_val.sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

# 將資料合併成dataset，方便之後匯入至模型
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_val.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
# download model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                            num_labels=len(encoding),
                            output_attentions=False,
                            output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [21]:
# 設定batch_size
batch_size = 32

# RandomSampler隨機抽出的資料
dataloader_train = DataLoader(dataset_train, 
                sampler=RandomSampler(dataset_train), 
                batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                  sampler=SequentialSampler(dataset_val), 
                  batch_size=batch_size)


In [22]:
from transformers import AdamW, get_linear_schedule_with_warmup
# Adam algorithm with weight decay fix 
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 3
# 設定learning rate變化的時程
scheduler = get_linear_schedule_with_warmup(optimizer, 
                       num_warmup_steps=0,
                       num_training_steps=len(dataloader_train)*epochs)

In [23]:
# 建立評估模型的function

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in encoding.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [24]:
# 將模型放到gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [26]:
# 使用驗證資料集評估模型
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':   batch[0],
              'attention_mask': batch[1],
              'labels':     batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [27]:
# 正式訓練模型
# tqdm優化輸出結果
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':   batch[0],
              'attention_mask': batch[1],
                  'labels': batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.8970176604092122
Validation loss: 0.31857325538756354
F1 Score (Weighted): 0.8994820933846382



Epoch 2
Training loss: 0.23855026403069496
Validation loss: 0.21476277768138854
F1 Score (Weighted): 0.921482238201608



Epoch 3
Training loss: 0.1592924412973225
Validation loss: 0.19467741156381274
F1 Score (Weighted): 0.9277727366271586



In [31]:
torch.save(model.state_dict(), '../content/finetuned_BERT_epoch_3.model')

In [40]:
df_test = pd.read_csv('../content/test.txt', 
            header = None, sep =';', names = ['sentence','label'],encoding='utf-8')

In [41]:
df_test['label'] = [encoding[key] for key in df_test.label]

encoded_data_test = tokenizer.batch_encode_plus(
    df_test.sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=256, 
    return_tensors='pt'
)

# 將資料合併成dataset，方便之後匯入至模型
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df_test.label.values)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [42]:
dataloader_test = DataLoader(dataset_test, 
                sampler=SequentialSampler(dataset_test), 
                batch_size=batch_size)

In [43]:
_, predictions, true_vals = evaluate(dataloader_test)

In [44]:
f1_score_func(predictions, true_vals)

0.9248742947815174

In [45]:
accuracy_per_class(predictions, true_vals)

Class: anger
Accuracy: 249/275

Class: fear
Accuracy: 207/224

Class: joy
Accuracy: 645/695

Class: love
Accuracy: 143/159

Class: sadness
Accuracy: 558/581

Class: surprise
Accuracy: 47/66



In [39]:
# load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                            num_labels=len(encoding),
                            output_attentions=False,
                            output_hidden_states=False)

model.to(device)
model.load_state_dict(torch.load('../content/finetuned_BERT_epoch_3.model', map_location=torch.device('cuda')))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>